In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark=(
    SparkSession
    .builder
    .appName("SparK first")
    .master("local[*]")
    .getOrCreate()
)
spark

# READING FILE 

In [24]:
df=spark.read.csv("C:\\Users\\Administrator\\Downloads\\final_data.csv",header=True,inferSchema=True,mode="permissive")

In [2]:
df=spark.read.csv("C:\\Users\\Administrator\\Downloads\\final_data.csv",header=True,inferSchema=True,mode="dropMalformed")

In [4]:
df.count()

19302

In [5]:
import os
os.getcwd()

'C:\\Users\\Administrator'

# REPRESENTING SAMPLE ROWS

In [18]:
df.show(5)

+---------------------+----------------+--------------------+--------------+------+-----------+------+------+--------------------+------------+-----+-----+-------+------------------+--------+--------------------+----------+--------------------+----------+------------------+-----------+--------+
|trans_date_trans_time|          cc_num|            merchant|      category|   amt|      first|  last|gender|              street|        city|state|  zip|    lat|              long|city_pop|                 job|       dob|           trans_num| unix_time|         merch_lat| merch_long|is_fraud|
+---------------------+----------------+--------------------+--------------+------+-----------+------+------+--------------------+------------+-----+-----+-------+------------------+--------+--------------------+----------+--------------------+----------+------------------+-----------+--------+
|  2019-08-11 19:38:33|4681601008538160|fraud_Hermann and...|  shopping_pos| 50.81|      Tammy| Davis|     F|776

# PRINTING SCHEMA

In [5]:
df.printSchema()

root
 |-- trans_date_trans_time: timestamp (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: double (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- unix_time: integer (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)



# SELECTING REQUIRED COLUMNS FOR FURTHER CALCULATIONS

In [10]:
is_fraud_df=df.select("first","last","merchant","city","category","amt","gender","dob","state","trans_date_trans_time","is_fraud").where(df.is_fraud==1)
is_fraud_df=is_fraud_df.distinct()

In [11]:
is_fraud_df.show()

+-----------+---------+--------------------+-------------+-------------+-------+------+----------+-----+---------------------+--------+
|      first|     last|            merchant|         city|     category|    amt|gender|       dob|state|trans_date_trans_time|is_fraud|
+-----------+---------+--------------------+-------------+-------------+-------+------+----------+-----+---------------------+--------+
|     Briana|   Santos|fraud_Towne, Gree...|   New Goshen| shopping_net| 905.28|     F|1960-01-06|   IN|  2019-01-15 15:15:18|       1|
|    Dorothy|   Thomas|fraud_Heidenreich...|      Milford|  grocery_pos|  301.8|     F|1996-11-12|   OH|  2019-01-22 00:04:52|       1|
|     Joseph|    Davis|fraud_Mosciski, Z...|     Nazareth| shopping_net|1146.99|     M|1980-07-30|   TX|  2019-02-03 23:19:40|       1|
|      Aaron|   Murray|fraud_Zieme, Bode...|    Meadville|gas_transport|   6.79|     M|1974-12-23|   MO|  2019-02-08 00:26:01|       1|
|       Lisa|Carpenter|fraud_Conroy-Crui...|    

In [6]:
is_fraud_df.count()

9651

# TOP FIVE CITIES WHERE MOST FRAUDULENET TRANSACTION OCCURES

In [12]:
top_five_cities=is_fraud_df.groupBy("city").count().orderBy("count",ascending=False).show(5)

+-------------+-----+
|         city|count|
+-------------+-----+
|       Dallas|   39|
|      Houston|   39|
|   Birmingham|   36|
|New York City|   35|
|       Camden|   34|
+-------------+-----+
only showing top 5 rows



# MOST fREQUENT CATEGORIES ASSOCIATED WITH FRAUDULENET TRANSACTIONS

In [13]:
top_categories=is_fraud_df.groupBy("category").count().orderBy("count",ascending=False).show(5)

+-------------+-----+
|     category|count|
+-------------+-----+
|  grocery_pos| 2228|
| shopping_net| 2219|
|     misc_net| 1182|
| shopping_pos| 1056|
|gas_transport|  772|
+-------------+-----+
only showing top 5 rows



# TOP TEN MOST FREQUENT MERCHANTS FOR FRAUDULANT TRANSACTIONS

In [14]:
top_ten_merchants=is_fraud_df.groupBy("merchant").count().orderBy("count",ascending=False).show(10)

+--------------------+-----+
|            merchant|count|
+--------------------+-----+
|   fraud_Kilback LLC|   62|
|  fraud_Rau and Sons|   60|
|   fraud_Kozey-Boehm|   60|
|     fraud_Doyle Ltd|   57|
|    fraud_Terry-Huel|   56|
|      fraud_Kuhn LLC|   55|
|     fraud_Boyer PLC|   55|
|     fraud_Kuhic LLC|   53|
|fraud_Moen, Reing...|   53|
|fraud_Kiehn-Emmerich|   53|
+--------------------+-----+
only showing top 10 rows



# AVERAGE AMOUNT SPEND BY EACH GENDER

In [14]:
average_expense=df.groupBy("gender").agg({"amt": "avg"}).orderBy("gender",ascending=False).show()

+------+------------------+
|gender|          avg(amt)|
+------+------------------+
|     M|330.50721562156156|
|     F|271.71298766157497|
+------+------------------+



# AVERAGE MERCHANT EXPENSE

In [126]:
average_merchant_expense=df.groupBy("merchant").agg({"amt": "avg"}).orderBy("merchant",ascending=False).show()

+--------------------+------------------+
|            merchant|          avg(amt)|
+--------------------+------------------+
|    fraud_Zulauf LLC|            18.785|
|fraud_Zieme, Bode...|11.623000000000001|
|fraud_Ziemann-Waters|            18.936|
|fraud_Zemlak, Til...|             18.91|
|  fraud_Zemlak Group| 790.3808333333335|
|fraud_Zboncak, Ro...| 993.9307894736845|
|   fraud_Zboncak Ltd|           113.332|
|   fraud_Zboncak LLC|          121.3175|
|   fraud_Yost-Rogahn|34.214285714285715|
|fraud_Yost, Scham...|             17.44|
|fraud_Yost, Block...|116.91428571428573|
|fraud_Wuckert-Walter|11.682500000000001|
|fraud_Wuckert-Gol...|246.58499999999998|
|fraud_Wuckert, Wi...|            264.89|
|      fraud_Wolf Inc| 312.7386666666667|
|fraud_Wiza, Schad...|29.354999999999997|
|      fraud_Wiza LLC|155.18714285714285|
|fraud_Witting, Be...|240.59333333333333|
|fraud_Wisozk and ...|200.31888888888892|
|fraud_Wintheiser,...|247.72142857142856|
+--------------------+------------

# TOP 5 STATES WITH HIGHEST NUMBER OF TRANSACTIONS

In [15]:
top_five_states=df.groupBy("state").count().orderBy("count",ascending=False).show(5)

+-----+-----+
|state|count|
+-----+-----+
|   NY| 1290|
|   TX| 1287|
|   PA| 1153|
|   CA|  857|
|   OH|  687|
+-----+-----+
only showing top 5 rows



In [17]:
from pyspark.sql.functions import year,month,current_date,to_date

# FINDING THE AGE OF ALL CUSTOMERS

In [63]:
all_users = is_fraud_df.select(
    "first", "last", "dob", "amt",
    (year(current_date()) - year(col("dob"))).alias("age")
)

distinct_users=all_users.select("first", "last","dob", "age").distinct().orderBy("first", "last")
distinct_users.count()
distinct_users.show()

+-------+--------+----------+---+
|  first|    last|       dob|age|
+-------+--------+----------+---+
|  Aaron|  Murray|1974-12-23| 51|
|  Aaron|    Pena|1950-11-27| 75|
|  Aaron|  Rogers|1945-03-15| 80|
|  Aaron| Stewart|1995-04-22| 30|
|   Adam|  Keller|1932-09-17| 93|
|   Adam|    Kirk|1931-09-12| 94|
|   Adam|Mcdonald|1991-01-31| 34|
|   Adam|  Riddle|1974-05-30| 51|
|   Adam|  Santos|1982-07-30| 43|
|   Adam|   Stark|1997-07-01| 28|
|Adriana|  Harvey|1950-09-15| 75|
|   Alan|   Colon|1993-09-11| 32|
|   Alan|  Howard|1946-11-01| 79|
|   Alan| Parsons|1955-12-04| 70|
|   Alan|  Taylor|1953-04-13| 72|
|   Alex|   Singh|1970-01-09| 55|
|   Alex|   White|1968-09-19| 57|
|  Alice|    Kemp|1985-04-15| 40|
| Alicia|  Bishop|1950-03-25| 75|
| Alicia| Hawkins|1933-03-15| 92|
+-------+--------+----------+---+
only showing top 20 rows



In [57]:
all_users.count()

9651

In [64]:
distinct_users.count()

976

# DISTRIBUTION OF TRANSACTION OVER THE COURSE OF THE MONTH

In [22]:
all_months=df.select((year("trans_date_trans_time").alias("transaction_year")),(month("trans_date_trans_time").alias("transaction_date"))).groupBy("transaction_year","transaction_date").count().orderBy("count",ascending=False).show()

+----------------+----------------+-----+
|transaction_year|transaction_date|count|
+----------------+----------------+-----+
|            2019|              12| 1293|
|            2020|              12| 1020|
|            2020|               6|  925|
|            2020|               5|  910|
|            2020|               8|  872|
|            2019|               3|  847|
|            2019|               8|  822|
|            2020|               3|  817|
|            2019|               6|  816|
|            2019|               9|  812|
|            2019|               1|  809|
|            2019|              10|  806|
|            2020|               7|  788|
|            2019|               7|  775|
|            2019|               5|  764|
|            2019|               2|  755|
|            2019|               4|  755|
|            2020|              10|  741|
|            2019|              11|  741|
|            2020|               9|  701|
+----------------+----------------

# RELATIONSHIP BETWEEN TRANSACTION DATE FRAUD OCCURANCE

In [29]:
all_dates=is_fraud_df.select((to_date("trans_date_trans_time")).alias("date")).groupBy("date").count().orderBy("date").show()

+----------+-----+
|      date|count|
+----------+-----+
|2019-01-02|    7|
|2019-01-03|   12|
|2019-01-04|   13|
|2019-01-05|   13|
|2019-01-06|    2|
|2019-01-07|    4|
|2019-01-08|   26|
|2019-01-09|   23|
|2019-01-10|    6|
|2019-01-11|   16|
|2019-01-12|   21|
|2019-01-13|   29|
|2019-01-14|   37|
|2019-01-15|   36|
|2019-01-16|   20|
|2019-01-17|   16|
|2019-01-18|   20|
|2019-01-19|   21|
|2019-01-20|   25|
|2019-01-21|   17|
+----------+-----+
only showing top 20 rows



In [67]:
from pyspark.sql.functions import when,col

In [69]:
all_users = all_users.withColumn(
    "age_group",
    when((all_users["age"] >= 18) & (all_users["age"] <= 25), "18-25")
    .when((all_users["age"] >= 26) & (all_users["age"] <= 35), "26-35")
    .when((all_users["age"] >= 36) & (all_users["age"] <= 45), "36-45")
    .when((all_users["age"] >= 46) & (all_users["age"] <= 55), "46-55")
    .when((all_users["age"] >= 56) & (all_users["age"] <= 65), "56-65")
    .when((all_users["age"] >= 66) & (all_users["age"] <= 75), "66-75")
    .when((all_users["age"] >= 76) & (all_users["age"] <= 85), "76-85")
    .when((all_users["age"] >= 86) & (all_users["age"] <= 95), "86-95")
    .otherwise("95+")
)

In [78]:
age_pattern=all_users.groupBy("age_group").agg({"amt": "avg"}).orderBy("age_group").show()

+---------+-----------------+
|age_group|         avg(amt)|
+---------+-----------------+
|    18-25| 606.911682692308|
|    26-35|530.0065985960447|
|    36-45|497.6876395348848|
|    46-55|461.6840605368364|
|    56-65|566.7271636953004|
|    66-75|576.2418645558481|
|    76-85| 576.577811671087|
|    86-95|537.2805781584578|
|      95+|563.1383163265306|
+---------+-----------------+

